# DMW 1 : Lab Report 1 Database

## Original Directories

### Yelp

In [1]:
#Files to be used: business (full), checkin (full), review (sample)
!ls -lh /mnt/data/public/yelp/20221007/yelp_dataset/

total 13G
-rw-r--r-- 1 3559 users  79K Feb 15  2022 Dataset_User_Agreement.pdf
-rw-r--r-- 1 3559 users 114M Jan 19  2022 yelp_academic_dataset_business.json
-rw-r--r-- 1 3559 users 274M Jan 19  2022 yelp_academic_dataset_checkin.json
-rw-r--r-- 1 3559 users 5.0G Jan 19  2022 yelp_academic_dataset_review.json
-rw-r--r-- 1 3559 users 173M Jan 19  2022 yelp_academic_dataset_tip.json
-rw-r--r-- 1 3559 users 3.2G Jan 19  2022 yelp_academic_dataset_user.json
-rw-rw-r-- 1 root root  4.1G Feb 24  2022 yelp_dataset.tgz


### Bookgraph

In [2]:
#Files to be used: books_romance (full), interactions_romance (sample)
!ls -lh /mnt/data/public/bookgraph

total 17G
-rw-rw-r-- 1 root root  37M Jul 11  2021 book_id_map.csv
-rw-rw-r-- 1 root root  18M Jul 11  2021 goodreads_book_authors.json.gz
-rw-rw-r-- 1 root root  98M Jul 11  2021 goodreads_books_children.json.gz
-rw-rw-r-- 1 root root  66M Jul 11  2021 goodreads_books_comics_graphic.json.gz
-rw-rw-r-- 1 root root  27M Jul 11  2021 goodreads_book_series.json.gz
-rw-rw-r-- 1 root root 266M Jul 11  2021 goodreads_books_fantasy_paranormal.json.gz
-rw-rw-r-- 1 root root 310M Jul 11  2021 goodreads_books_history_biography.json.gz
-rw-rw-r-- 1 root root 2.0G Jul 11  2021 goodreads_books.json.gz
-rw-rw-r-- 1 root root 220M Jul 11  2021 goodreads_books_mystery_thriller_crime.json.gz
-rw-rw-r-- 1 root root  27M Jul 11  2021 goodreads_books_poetry.json.gz
-rw-rw-r-- 1 root root 348M Jul 11  2021 goodreads_books_romance.json.gz
-rw-rw-r-- 1 root root 100M Jul 11  2021 goodreads_books_young_adult.json.gz
-rw-rw-r-- 1 root root  72M Jul 11  2021 goodreads_book_works.json.gz
-rw-rw-r-- 1 root root 4

## Database Creation

In [2]:
import sqlite3
import pandas as pd
import json
import db_caller

In [3]:
# Get paths
db_path = db_caller.connect("LR1.db", path_only=True)
yelp_path = "/mnt/data/public/yelp/20221007/yelp_dataset/"
book_path = "/mnt/data/public/bookgraph/"

In [4]:
conn = sqlite3.connect(db_path)

### Yelp Data

#### Yelp: Checkin Data

In [6]:
# Read json file as DataFrame
checkin_df = pd.read_json(yelp_path + "yelp_academic_dataset_checkin.json",
                          lines=True)

In [8]:
# Add 'yelp_checkin' to database if not already there
try:
    checkin_df.to_sql('yelp_checkin',
                      conn, if_exists='fail', index=False)
except ValueError:
    print('Already in database.')

Already in database.


In [7]:
# Reset variable
checkin_df = None

#### Yelp: Business Data

In [10]:
# Read json file as json object
content = []
for line in open(yelp_path + "yelp_academic_dataset_business.json", 'r'):
    content.append(json.loads(line))

In [11]:
# Read json object as a normalized DataFrame
business_df = pd.json_normalize(content)

In [12]:
# Add 'yelp_business' to database if not already there
try:
    business_df.to_sql('yelp_business',
                       conn, if_exists='fail', index=False)
except ValueError:
    print('Already in database.')

Already in database.


In [ ]:
# Reset variables
business_df = None
content = None

#### Yelp: Review Data (Sample)

In [5]:
# Read json file as DataFrame every 1 million lines
rev_dfs = []
rev_chunk = pd.read_json(yelp_path + "yelp_academic_dataset_review.json",
                         lines=True, chunksize=1_000_000)

**WARNING:** The following cell will take a long time to execute

In [6]:
# Get a random sample of 50,000 for every 1 million lines (~5% of total data)
for rev_df in rev_chunk:
    rev_dfs.append(rev_df.sample(50_000))

In [12]:
# Combine DataFrame chunks into 1
review_df = pd.concat(rev_dfs)

In [13]:
# Add 'yelp_review' to database if not already there
try:
    review_df.to_sql('yelp_review',
                     conn, if_exists='fail', index=False)
except ValueError:
    print('Already in database.')

In [14]:
# Reset variables
rev_dfs = None
rev_df = None
rev_chunk = None
review_df = None

### Bookgraph Data

#### Bookgraph: Romance Books Data

In [11]:
# Read json file as DataFrame every 10k lines
rombooks_dfs = []
rombooks_chunks = pd.read_json(book_path + "goodreads_books_romance.json.gz",
                               lines=True, compression="gzip",
                               chunksize=10_000)

In [12]:
# Get only the specified columns
features = ["book_id",
            "title",
            "ratings_count",
            "country_code",
            "publication_year",
            "publication_month",
            "publication_day",
            "is_ebook"]

for rombooks_df in rombooks_chunks:
    rombooks_df = rombooks_df[features]
    rombooks_dfs.append(rombooks_df)

In [13]:
# Combine DataFrame chunks into 1
romance_books_df = pd.concat(rombooks_dfs)

In [18]:
# Add 'bookgraph_romance_books' to database if not already there
try:
    romance_books_df.to_sql('bookgraph_romance_books',
                            conn, if_exists='fail', index=False)
except ValueError:
    print('Already in database.')

Already in database.


In [ ]:
# Reset variables
rombooks_dfs = None
rombooks_chunks = None
features = None
rombooks_df = None
romance_books_df = None

#### Bookgraph: Romance Book Interactions Data

In [72]:
# Read json file as DataFrame every 1 million lines
rom_dfs = []
rom_chunk = pd.read_json(book_path + "goodreads_interactions_romance.json.gz",
                         lines=True, compression="gzip", chunksize=1_000_000)

**WARNING:** The following cell will take a long time to execute

In [73]:
# Get a random sample of 50,000 for every 1 million lines (~5% of total data)
for rom_df in rom_chunk:
    rom_dfs.append(rom_df.sample(50_000))

In [22]:
# Add 'bookgraph_romance_interactions' to database if not already there
try:
    romance_interactions_df.to_sql('bookgraph_romance_interactions',
                                   conn, if_exists='fail', index=False)
except ValueError:
    print('Already in database.')

Already in database.


In [84]:
# Reset variables
rom_dfs = None
rom_df = None
rom_chunk = None
romance_interactions_df = None